In [1]:
import numpy as np
import json
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import tensorflow as tf

In [2]:
len_vcb = 366549
seed = 89
max_len = 150

In [3]:
with open('/home/rafael/Documents/Faculdade/ICs/IC Direito/codes/pre-processing/3. Fine-Tuning NER rodando no servidor/Resultados em toda base de SP/utils/tokenizer_100k.json') as f: 
    data = json.load(f) 
    tok = tokenizer_from_json(data)

In [4]:
### definindo metricas customizadas

def recall(y_true, y_pred):
    y_pred = tf.cast(K.argmax(y_pred), tf.float32) # transformando o vetor de (None, 150, n_tags) para (None, 150)
    macro_recall = 0
    y_pred = y_pred[y_true != 0] # retira zero
    y_true = y_true[y_true != 0] # retira zero
    equals = y_pred[K.equal(y_true,y_pred)]  # calcula os verdadeiros positivos, ignorando o zero
    
    for i in range(1,n_tags):
        equals_positives = equals[equals == i]  
        true_positives = K.sum(equals_positives/i) # divide por i, para que os valores se tornem 1

        equals_possibles = y_true[K.equal(y_true,K.constant(i))]/i
        possible_positives = K.sum(equals_possibles)

        recall = true_positives / (possible_positives + K.epsilon())
        macro_recall = macro_recall + recall

    macro_recall = macro_recall/(n_tags-1)  

    return macro_recall

def precision(y_true, y_pred):
    y_pred = tf.cast(K.argmax(y_pred), tf.float32) # trannsformando o vetor de (None, 150, n_tags) para (None, 150)
    macro_precision = 0
    y_pred = y_pred[y_true != 0] # retira zero
    y_true = y_true[y_true != 0] # retira zero
    equals = y_true[K.equal(y_true,y_pred)]  # calcula os verdadeiros positivos, ignorando o zero
    
    for i in range(1,n_tags):
        equals_positives = equals[equals == i]  
        true_positives = K.sum(equals_positives/i) # divide por i, para que os valores se tornem 1

        equals_pred_possibles = y_pred[K.equal(y_pred,K.constant(i))]/i
        predicted_positives = K.sum(equals_pred_possibles)

        precision = true_positives / (predicted_positives + K.epsilon())
        macro_precision = macro_precision + precision

    macro_precision = macro_precision/(n_tags-1)

    return macro_precision

def macro_f1(y_true, y_pred):
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2*((prec*rec)/(prec+rec+K.epsilon()))

In [5]:
custom_metrics = {"recall": recall,"precision": precision, "macro_f1": macro_f1}
model = tf.keras.models.load_model(
    '/home/rafael/Documents/Faculdade/ICs/IC Direito/codes/pre-processing/3. Fine-Tuning NER rodando no servidor/Resultados em toda base de SP/SP_TOTAL_1EPOCH.h5',
    custom_objects=custom_metrics)

In [6]:
são_paulo_df = pd.read_json('/home/rafael/Documents/Faculdade/ICs/IC Direito/codes/pre-processing/SP_PROCESSO_E_ASSUNTO_FEITOS_BCKPV2.json')    

In [7]:
são_paulo_sample = são_paulo_df.sample(1, random_state=seed)['Conteúdo'].to_list()
são_paulo_sample

['Processo 1001933-38.2015.8.26.0269 - Execução de Alimentos - Valor da Execução / Cálculo / Atualização - I.M.E.L.S. - A.L.S. - Vistos.Fls. 245/246 e 248: Ciente.Fls. 249/250: Em que o apresentado, tais argumentos carecem de comprovação. De tal modo, para tanto, concedo 10 (dez) dias.Após, com ou sem manifestação, tornem conclusos.Int. - ADV: FÁBIO ALBUQUERQUE (OAB 164311/SP), VITOR DE CAMARGO HOLTZ MORAES (OAB 134223/SP), MARINA LEMBO TEDESCHI LÊRA PALMIRO (OAB 364785/SP)']

In [8]:
são_paulo_sample_tokenized = tok.texts_to_sequences(são_paulo_sample)
pd.DataFrame(são_paulo_sample_tokenized[0]).T

,0,1,2,3,4,5,6,7,8,9,...,71,72,73,74,75,76,77,78,79,80
0,16,68804,755,90,15,17,3932,78,2,290,...,46783,11,2755,81592,15332,203045,133706,10,203046,11


In [9]:
são_paulo_sample_tokenized_padded = pad_sequences(maxlen=max_len,
                                    sequences=são_paulo_sample_tokenized,
                                    padding="post",
                                    truncating="post")
pd.DataFrame(são_paulo_sample_tokenized_padded)

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,16,68804,755,90,15,17,3932,78,2,290,...,0,0,0,0,0,0,0,0,0,0


In [10]:
predicted_são_paulo_sample_tokenized_padded = model(são_paulo_sample_tokenized_padded)

In [11]:
for i,p in enumerate(predicted_são_paulo_sample_tokenized_padded[0]):
    if (i < len(são_paulo_sample_tokenized[0])): 
        print(np.argmax(p),tok.index_word[são_paulo_sample_tokenized[0][i]])

1 Processo
1 1001933
1 38
1 2015
1 8
1 26
1 0269
14 Execução
2 de
2 Alimentos
1 Valor
1 da
1 Execução
1 Cálculo
1 Atualização
1 I
1 M
1 E
1 L
1 S
1 A
1 L
1 S
1 Vistos
1 Fls
1 245
1 246
1 e
1 248
1 Ciente
1 Fls
1 249
1 250
1 Em
1 que
1 o
1 apresentado
1 tais
1 argumentos
1 carecem
1 de
1 comprovação
1 De
1 tal
1 modo
1 para
1 tanto
1 concedo
1 10
1 dez
1 dias
1 Após
1 com
1 ou
1 sem
1 manifestação
1 tornem
1 conclusos
1 Int
1 ADV
1 FÁBIO
1 ALBUQUERQUE
1 OAB
1 164311
1 SP
1 VITOR
1 DE
1 CAMARGO
1 HOLTZ
1 MORAES
1 OAB
1 134223
1 SP
1 MARINA
1 LEMBO
1 TEDESCHI
1 LÊRA
1 PALMIRO
1 OAB
1 364785
1 SP
